# Notebook 02 — Feature Engineering (Silver & Gold Layers)

This notebook performs feature transformations required for machine learning:
- Load Bronze table
- Identify numerical and categorical columns
- Apply StringIndexer + OneHotEncoder
- Normalize numerical features
- Assemble final feature vector
- Save Silver and Gold tables


In [0]:
df_bronze = spark.read.format("delta").load(
    "dbfs:/Volumes/workspace/credit-risk/credit-risk/bronze"
)

df_bronze.display()


In [0]:
categorical_cols = [c for c, t in df_bronze.dtypes if t == "string"]
numeric_cols = [c for c, t in df_bronze.dtypes if t in ["int", "double"]]

categorical_cols, numeric_cols